In [14]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io
from concurrent.futures import ThreadPoolExecutor, as_completed

In [15]:
from azure.storage.blob import BlobServiceClient
import pandas as pd
import io
import pyarrow.parquet as pq
from concurrent.futures import ThreadPoolExecutor, as_completed

# SAS Token and Container URL
sas_token = "sp=racwdl&st=2024-12-01T01:41:21Z&se=2025-01-28T09:41:21Z&spr=https&sv=2022-11-02&sr=c&sig=OpjDBOtKtITxiEAnzF7wkFAvAyLCmijQLyz93sxQi78%3D"
container_url = "https://datalakestoragerentscape.blob.core.windows.net"

# Initialize BlobServiceClient
blob_service_client = BlobServiceClient(account_url=container_url, credential=sas_token)

# Get ContainerClient
container_client = blob_service_client.get_container_client(container="openai-rentscape-blob")

# Function to download blob and load it into a DataFrame
def download_and_load_blob(blob_name):
    try:
        blob_client = container_client.get_blob_client(blob_name)
        blob_data = blob_client.download_blob().readall()
        
        # Determine file type and load appropriately
        if blob_name.endswith('.csv'):
            df = pd.read_csv(io.StringIO(blob_data.decode('utf-8')))
        elif blob_name.endswith('.parquet'):
            df = pd.read_parquet(io.BytesIO(blob_data))
        else:
            print(f"Skipping unsupported file type: {blob_name}")
            return
        
        # Generate a variable name by removing the file extension
        variable_name = blob_name.rsplit('.', 1)[0].replace('/', '_').replace('-', '_').replace(' ', '_')
        
        # Set the DataFrame to a global variable with this name
        globals()[variable_name] = df
        print(f"Loaded {blob_name} into DataFrame: {variable_name}")
    except Exception as e:
        print(f"Failed to process {blob_name}: {e}")

# List all blobs in the container
blob_names = [blob.name for blob in container_client.list_blobs()]

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=10) as executor:
    # Submit tasks for each blob
    futures = {executor.submit(download_and_load_blob, blob_name): blob_name for blob_name in blob_names}
    
    # Process results as they complete
    for future in as_completed(futures):
        future.result()  # Ensures all processing completes

# After this, each file in the blob will be loaded as a DataFrame
# The variable names will correspond to the file names (excluding extensions).


Loaded openai-rentscape-blob/barcelona_listings.parquet into DataFrame: openai_rentscape_blob_barcelona_listings
Loaded openai-rentscape-blob/barcelona_neighborhoods_details.csv into DataFrame: openai_rentscape_blob_barcelona_neighborhoods_details
Loaded openai-rentscape-blob/prague_neighborhoods_details.csv.csv into DataFrame: openai_rentscape_blob_prague_neighborhoods_details.csv
Loaded openai-rentscape-blob/prague_listings.parquet into DataFrame: openai_rentscape_blob_prague_listings


In [16]:
# List all variables in the global namespace
dataframes = {name: obj for name, obj in globals().items() if isinstance(obj, pd.DataFrame)}

# Print the names of all loaded DataFrames
print("Loaded DataFrames:")
for df_name in dataframes.keys():
    print(df_name)


Loaded DataFrames:
openai-rentscape-blob_barcelona_neighborhoods_details_csv
openai-rentscape-blob_prague_neighborhoods_details_csv_csv
openai-rentscape-blob_barcelona_listings_parquet
openai-rentscape-blob_prague_listings_parquet
openai_rentscape_blob_prague_neighborhoods_details.csv
openai_rentscape_blob_barcelona_neighborhoods_details
openai_rentscape_blob_prague_listings
openai_rentscape_blob_barcelona_listings


In [17]:
openai_rentscape_blob_barcelona_listings

,id,openai_classification
0,18674,Near major landmarks; Family-friendly; Quiet a...
1,23197,Central location; Family-friendly; Spacious li...
2,32711,Near major landmarks; Family-friendly; Cozy at...
3,34241,Central location; Close to cultural attraction...
4,34981,Spacious living areas; Family-friendly; Fully ...
...,...,...
19477,1234621063113396232,Central location; Fully equipped kitchen; Luxu...
19478,1234633310288552263,Central location; Ideal for couples; Luxury ap...
19479,1234636058660290258,Central location; Fully equipped kitchen; Luxu...
19480,1234659029532006287,Cozy atmosphere; Includes cleaning service; Id...
